<a href="https://colab.research.google.com/github/i-m-vivek/Deep-Learning-Models/blob/master/Arcnet_in_TF2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==2.0.0
tf.__version__

     |████████████████████████████████| 86.3MB 46kB/s 
     |████████████████████████████████| 3.8MB 47.3MB/s 
     |████████████████████████████████| 450kB 49.0MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

'1.15.0'

In [1]:
import tensorflow as tf 
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Reshape , Dense , Lambda ,Add ,Multiply , dot, Flatten 
from keras.layers import GlobalAveragePooling1D, Dropout, BatchNormalization
from keras.layers import Layer
from keras.models import Model
import numpy as np

tf.__version__

Using TensorFlow backend.


'2.0.0'

In [0]:
configs = {
    "num_units_rnn1" : 64, 
    "num_units_rnn2" : 64, 
    "num_units_rnn3" : 128, 
    "num_recurrence" : 30,
    "num_lstm" : 3, 
    "batch_size": 62,
    "epochs" : 50
}

In [3]:
inputs = tf.keras.layers.Input(shape = (256, 256, 3))
out = tf.keras.applications.vgg16.VGG16(include_top= False)(inputs)
encoder_out = tf.keras.layers.Reshape((64, 512))(out)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
h1 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn1"]])
h2 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn2"]])
h3 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn3"]])

c1 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn1"]])
c2 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn2"]])
c3 = tf.random.uniform([configs["batch_size"], configs["num_units_rnn3"]])

out2attn = tf.keras.layers.Dense(64, activation = "softmax")

rnn1 = tf.keras.layers.LSTM(configs["num_units_rnn1"], return_sequences=False,return_state=True)
rnn2 = tf.keras.layers.LSTM(configs["num_units_rnn2"], return_sequences=False,return_state=True)
rnn3 = tf.keras.layers.LSTM(configs["num_units_rnn3"], return_sequences=False,return_state=True)

for i in range(configs["num_recurrence"]):
    out_of_h3_2attn =  out2attn(h3)
    out_h3_attn = tf.keras.layers.Reshape((64, 1))(out_of_h3_2attn)
    out_h3_attn = tf.math.multiply(out_h3_attn, encoder_out)

    _, final_memory_state1, final_carry_state1 = rnn1(out_h3_attn, initial_state = [h1, c1])
    final_memory_state1_new = tf.expand_dims(final_memory_state1, 1)
    _, final_memory_state2, final_carry_state2 = rnn2(final_memory_state1_new,  initial_state = [h2, c2])
    final_memory_state2_new = tf.expand_dims(final_memory_state2, 1)
    _, final_memory_state3, final_carry_state3 = rnn3(final_memory_state2_new,  initial_state = [h3, c3])

    h1, c1 = final_memory_state1, final_carry_state1
    h2, c2 = final_memory_state2, final_carry_state2
    h3, c3 = final_memory_state3, final_carry_state3

In [0]:
out_last = tf.keras.layers.Dense(31, activation = "softmax")(final_memory_state3)

In [0]:
model = tf.keras.Model(inputs, out_last)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 64, 512)      0           vgg16[1][0]                      
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(62, 64, 512)]      0           reshape[0][0]                    
______________________________________________________________________________________________

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
!unzip "/content/drive/My Drive/optimal31.zip"

Archive:  /content/drive/My Drive/optimal31.zip
 extracting: content/data/val/freeway/freeway(58).jpg  
 extracting: content/data/val/freeway/freeway(7).jpg  
 extracting: content/data/val/freeway/freeway(16).jpg  
 extracting: content/data/val/freeway/freeway(22).jpg  
 extracting: content/data/val/freeway/freeway(3).jpg  
 extracting: content/data/val/freeway/freeway(9).jpg  
 extracting: content/data/val/freeway/freeway(42).jpg  
 extracting: content/data/val/freeway/freeway(45).jpg  
 extracting: content/data/val/freeway/freeway(52).jpg  
 extracting: content/data/val/freeway/freeway(50).jpg  
 extracting: content/data/val/freeway/freeway(25).jpg  
 extracting: content/data/val/freeway/freeway(37).jpg  
 extracting: content/data/val/freeway/freeway(44).jpg  
 extracting: content/data/val/freeway/freeway(60).jpg  
 extracting: content/data/val/freeway/freeway(23).jpg  
 extracting: content/data/val/freeway/freeway(46).jpg  
 extracting: content/data/val/freeway/freeway(20).jpg  
 ex

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True, 
    vertical_flip=True)

val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory("/content/content/data/train",
                                                    target_size = (256, 256),
                                                    batch_size= 62,
                                                    shuffle= True)
val_generator = train_datagen.flow_from_directory("/content/content/data/val",
                                                    target_size = (256, 256),
                                                    batch_size= 62,
                                                    shuffle= True)

Found 1302 images belonging to 31 classes.
Found 558 images belonging to 31 classes.


In [0]:
model.compile(loss= "categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [0]:
model.fit_generator(train_generator, steps_per_epoch=1302 // 62, 
                    epochs= 50, validation_data= val_generator,
                    validation_steps= 558//62)

Epoch 1/50
 3/21 [===>..........................] - ETA: 49:38 - loss: 3.4358 - accuracy: 0.0430